In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
### Imports
import os
import sys

import numpy as np
import pandas as pd

import pickle
import cv2

from pathlib import Path
from tensorflow.keras.models import Model

# import supported models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.compat.v1.keras.models import load_model
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import Adam

In [32]:
# get project path, and flag if script runs in a file
project_path = Path(os.getcwd(), os.pardir)
output_path = project_path / "output"

# add project root to pythonpath
sys.path.insert(0, str(project_path / "src"))

# import custom packages
from utils.identify import *   
from utils.findsets import *   
from set_cardgame.dataset import *

In [33]:
# name of first layer that had 100% validation score  
layer_name = "res3d_branch2a"
intermediate_model = load_model(
    output_path / f"resnet_{layer_name}_intermediate_model.h5",
    custom_objects=None,
    compile=True
)

### load model

In [34]:
nr_images = 8100
layer_name = "res3d_branch2a"

with open(output_path / f'clf_{layer_name}_{nr_images}.pkl', 'rb') as handle:
    best_model = pickle.load(handle)

In [35]:
# load training and validation data
X_test, y_test, X_val, y_val = load_dataset(nr_images=810, output_path=None, preprocessing_func=preprocess_input_resnet50)
print("X_test.shape:", X_test.shape)

Found 243 images belonging to 81 classes.
Found 81 images belonging to 81 classes.
X_test.shape: (810, 96, 128, 3)


In [7]:
# load meta data about the dataset
df_meta, _ = load_metadata()

In [8]:
# generate image embeddings
X_test_embeddings = intermediate_model.predict(X_test).reshape(X_test.shape[0], -1)

In [9]:
test_score = best_model.score(X_test_embeddings, y_test)
print(f"test_score:{test_score}")

test_score:1.0


In [71]:
# todo: add support in dataset.py to quickly get the label and catories

def get_card_meta(df_meta, card_id):
    return df_meta[(df_meta.card_id == card_id)].drop_duplicates()

def get_feature_codes(df_meta, card_id):
    card_feature_codes = []
    print("card_id:", card_id)
    for feature in ['color', 'shape', 'fill', 'number']:
        feature_categories = pd.Categorical(df_meta[feature])
        df_meta[f'{feature}_code'] = feature_categories.codes
    
    return card_feature_codes

def get_card_label(df_meta, card_id):
    return " ".join(list(df_meta[df_meta.card_id == int(card_id)][['color', 'shape', 'fill', 'number']].drop_duplicates().values[0]))

# print(get_feature_codes(df_meta, 49))
# print(get_card_label(df_meta, 49))

In [72]:
get_card_meta(df_meta, 49)

,filename,card_id,variant,color,shape,fill,number,color_code,shape_code,fill_code,number_code
196,0_green_square_dotted_two.jpg,49,0,green,square,dotted,two,1,0,2,1
197,1_green_square_dotted_two.jpg,49,1,green,square,dotted,two,1,0,2,1
198,2_green_square_dotted_two.jpg,49,2,green,square,dotted,two,1,0,2,1
199,3_green_square_dotted_two.jpg,49,3,green,square,dotted,two,1,0,2,1


### Detect SET combinations

In [18]:
from utils.findsets import findsets
collection = [
    (1, 1, 1, 1),
    (2, 2, 2, 2),
    (3, 3, 3, 3),
    (1, 2, 3, 1),
    (3, 1, 1, 2),
    (2, 1, 2, 2),
    (3, 3, 1, 3)
]
findsets(np.array(collection))

[[array([0]), array([1]), array([2])], [array([3]), array([5]), array([6])]]

In [25]:
np.array(list(map(lambda x: get_feature_codes(df_meta, x), predictions)))

card_id: 49
card_id: 77
card_id: 74
card_id: 54
card_id: 5
card_id: 80
card_id: 21
card_id: 9
card_id: 10
card_id: 23
card_id: 22
card_id: 32


array([[2, 2, 0, 0],
       [2, 1, 2, 2],
       [2, 1, 2, 0],
       [2, 2, 0, 2],
       [2, 0, 2, 2],
       [2, 1, 2, 1],
       [2, 0, 0, 1],
       [2, 0, 2, 1],
       [2, 0, 2, 1],
       [2, 0, 0, 1],
       [2, 0, 0, 1],
       [2, 0, 1, 1]], dtype=int8)

In [20]:
predictions_features

array([[2, 2, 0, 0],
       [2, 1, 2, 2],
       [2, 1, 2, 0],
       [2, 2, 0, 2],
       [2, 0, 2, 2],
       [2, 1, 2, 1],
       [2, 0, 0, 1],
       [2, 0, 2, 1],
       [2, 0, 2, 1],
       [2, 0, 0, 1],
       [2, 0, 0, 1],
       [2, 0, 1, 1]], dtype=int8)

In [19]:
findsets(predictions_features)

[[array([0]), array([1]), array([11])],
 [array([1]), array([2]), array([5])],
 [array([2]), array([3]), array([11])],
 [array([ 6,  9, 10]), array([7, 8]), array([11])],
 [array([ 6,  9, 10]), array([7, 8]), array([11])],
 [array([ 6,  9, 10]), array([ 6,  9, 10]), array([ 6,  9, 10])],
 [array([7, 8]), array([ 6,  9, 10]), array([11])],
 [array([7, 8]), array([ 6,  9, 10]), array([11])],
 [array([7, 8]), array([ 6,  9, 10]), array([11])],
 [array([7, 8]), array([ 6,  9, 10]), array([11])]]

In [15]:
# putting it all together
test_images_path = project_path / "test_images"

#X_identify = np.array([])
target_width = 128
target_height = 96
for image_name in os.listdir(test_images_path):
        
    # read image
    img = cv2.imread(str(test_images_path / image_name))

    # identify the different cards
    identified_images, bboxes = identify_images(img, target_size=(target_width, target_height))
    
    # generate image embeddings for each card
    image_embeddings = intermediate_model.predict(identified_images).reshape(identified_images.shape[0], -1)
    print(f'image_embeddings.shape: {image_embeddings.shape}')
    
    # classify cards
    predictions = best_model.predict(image_embeddings)

    # get feature codes for each prediction
    predictions_features = np.array(list(map(lambda x: get_feature_codes(df_meta, x), predictions)))
    print("predictions_features:", predictions_features)
    # find all sets
    sets = findsets(a)
    print(sets)
    # plot sets
    #plot_sets(img_rgb, image_name, bboxes, sets)

    # plot predictions
    #plot_predictions(image_name, identified_images, predictions2str(predictions))

image_embeddings.shape: (12, 24576)
predictions_features: [[2 2 0 0]
 [2 1 2 2]
 [2 1 2 0]
 [2 2 0 2]
 [2 0 2 2]
 [2 1 2 1]
 [2 0 0 1]
 [2 0 2 1]
 [2 0 2 1]
 [2 0 0 1]
 [2 0 0 1]
 [2 0 1 1]]


NameError: name 'a' is not defined

In [44]:
" ".join(df_meta[df_meta.card_id == int(card_id)][['color', 'shape', 'fill', 'number']].drop_duplicates().values[0])

'purple square dotted three'

### read from camera

In [12]:
cap = cv2.VideoCapture(0)

while(1):
    ret, frame = cap.read()
    
    cv2.imshow('Camera', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()